In [1]:
# En caso de no tenerlo
# % pip install mlflow

In [1]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import joblib

Se usan datos sinteticos para probar el modelo.

In [ ]:
df = pd.read_csv('/home/project/data/simulados/simulados.csv')
df = df.drop(['TIPO_PACIENTE',], axis=1)
scaler = joblib.load('/home/project/models/preprocess_utils/scaler.pkl')
encoder = joblib.load('/home/project/models/preprocess_utils/encoder.pkl')
encoded_columns  = encoder.transform(df[['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']])
scaled_columns = scaler.fit_transform(df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1))
encoded_scaled_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']))
scaled_df = pd.DataFrame(scaled_columns, columns=df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1).columns)
data = pd.concat([encoded_scaled_df, scaled_df], axis=1)

Configurar MLFlow para acceder al servidor de Dagshub que contiene los modelos

En Dagshub esta alojado el modelo, y para que MLFLow pueda acceder a el se le tiene que indicar de la siguiente forma.

In [3]:
mlflow.set_tracking_uri("https://dagshub.com/empleo-inducido/project-ml.mlflow")

Dentro de variables `model_name` y `model_version` se guardan como estan etiquedos en MLFLow.

Se tiene que indicar que busque uno de los modelos guardados por su nombre, además de indicar su version. Con el nombre 'latest' se ira por la ultima version guardada del modelo.

In [4]:
model_name = "testing-model"
model_version = 'latest'

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

El modelo ya esta guardado y se puede hace predicción con los datos de muestra.

In [19]:
model.predict(data)

array([1, 1, 1, 1, 1, 1])